In [ ]:
!pip install Sastrawi

     |████████████████████████████████| 209 kB 5.1 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from tensorflow.keras import layers, initializers
from tensorflow.keras.layers import Concatenate, Input, Embedding, LSTM, Attention, Dense, Dropout, Bidirectional, LSTM, Flatten, Conv1D, GlobalMaxPool1D, MaxPool1D, Reshape, GlobalAveragePooling1D, UpSampling1D, Layer

from subprocess import check_output

import tensorflow
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
import datetime
import sys
import os
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs

sns.set_style("whitegrid")
np.random.seed(0)
nltk.download('punkt')

tokenizer = RegexpTokenizer(r'\w+')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_df = pd.read_csv('/content/sicepat-train.csv', sep=',', header=0)
test_df = pd.read_csv('/content/sicepat-test.csv', sep=',', header=0)

print("num train: ", train_df.shape[0])
print("num test: ", test_df.shape[0])

num train:  550
num test:  98


In [ ]:
raw_docs_train = train_df['text'].tolist()
raw_docs_test = test_df['text'].tolist() 
num_classes = 3

In [ ]:
# preprocessing train
print("pre-processing train data...")
processed_docs_train = []
for doc in tqdm(raw_docs_train):
    doc = re.sub(r'\b(halaman|halaman all|all)\b', '', doc)
    doc = re.sub(r'\b(com|co id|net|id|news)\b', '', doc)
    doc = re.sub(r'\b(news)\b', '', doc)
    doc = re.sub(r'\b(tribun|tribunnews|fimela)', '', doc)
    tokens = tokenizer.tokenize(doc)
    #filtered = [word for word in tokens if word not in data]
    processed_docs_train.append(doc)

pre-processing train data...


100%|██████████| 550/550 [00:00<00:00, 26335.00it/s]


In [ ]:
np.savetxt('train-sicepat.txt', processed_docs_train, delimiter=' ', fmt="%s")
! wc train-sicepat.txt
! head -n 9737 train-sicepat.txt > sicepat.train

  550 11833 80730 train-sicepat.txt


In [ ]:
# preprocessing test
processed_docs_test = []
for doc in tqdm(raw_docs_test):
    doc = re.sub(r'\b(halaman|halaman all|all)\b', '', doc)
    doc = re.sub(r'\b(com|co id|net|id|news)\b', '', doc)
    doc = re.sub(r'\b(news)\b', '', doc)
    doc = re.sub(r'\b(tribun|tribunnews|fimela)', '', doc)
    tokens = tokenizer.tokenize(doc)
    #filtered = [word for word in tokens if word not in data]
    processed_docs_test.append(doc)

100%|██████████| 98/98 [00:00<00:00, 20837.56it/s]


In [ ]:
np.savetxt('test-sicepat.txt', processed_docs_test, delimiter=' ', fmt="%s")
! wc test-sicepat.txt
! head -n 900 test-sicepat.txt > sicepat.test

   98  1976 13507 test-sicepat.txt


In [ ]:
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
!unzip v0.9.2.zip
%cd fastText-0.9.2
!make
!pip install .

--2022-01-10 15:10:22--  https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2 [following]
--2022-01-10 15:10:23--  https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v0.9.2.zip’

v0.9.2.zip              [  <=>               ]   4.17M  16.5MB/s    in 0.3s    

2022-01-10 15:10:23 (16.5 MB/s) - ‘v0.9.2.zip’ saved [4369852]

Archive:  v0.9.2.zip
5b5943c118b0ec5fb9cd8d20587de2b2d3966dfe
   creating: fastText-0.9.2/
   creating: fastText-0.9.2/.circleci/
  inflating: fastText-0

In [ ]:
import fasttext

training_data_path ='/content/sicepat.train' 
validation_data_path ='/content/sicepat.test'
model_path ='/content/'
model_name="model-headline"
vec_docs = "/content/drive/fasttext/wiki.id.vec"

def train():
    print('Training start')
    try:
        hyper_params = {"lr": 0.008,
                        "epoch": 10,
                        "wordNgrams": 4,
                        "dim": 300,
                        "pretrainedVectors": vec_docs
                        }     
                               
        print(str(datetime.datetime.now()) + ' START=>' + str(hyper_params) )

        # Train the model.
        global model
        model = fasttext.train_supervised(input=training_data_path, **hyper_params)
        print("Model trained with the hyperparameter \n {}".format(hyper_params))

        # CHECK PERFORMANCE
        print(str(datetime.datetime.now()) + 'Training complete.' + str(hyper_params) )
        
        result = model.test(training_data_path)
        validation = model.test(validation_data_path)
        
        # DISPLAY ACCURACY OF TRAINED MODEL
        text_line = str(hyper_params) + ",accuracy:" + str(result[1])  + ",validation:" + str(validation[1]) + '\n' 
        print(text_line)
        
        #quantize a model to reduce the memory usage
        model.quantize(input=training_data_path, qnorm=True, retrain=True, cutoff=100000)
        print("Model is quantized!!")
        model.save_model(os.path.join(model_path,model_name + ".ftz"))                
    
        #TESTING        
        print(model.predict(['paket gw dibobol dong'],k=3)) #neg
        print(model.predict(['aduh ekspedisi ruwet banget'],k=1)) #neg
        print(model.predict(['kurir baik'], k=1)) #pos
        print(model.predict(['ongkir ekspedisi berapa min'], k=1)) #neu
        print(model.predict(['cek dm dong elah min'], k=1)) #neu
        print(model.predict(['eh udah nyampe, cepet bener dah'], k=1)) #neg
        
        
    except Exception as e:
        print('Exception during training: ' + str(e) )


# Train your model.
train()

Training start
2022-01-10 15:44:51.972375 START=>{'lr': 0.008, 'epoch': 10, 'wordNgrams': 4, 'dim': 300, 'pretrainedVectors': '/content/drive/MyDrive/College/Careers/Accepted Job/04 Solusi247 - Data Scientist/Jobs/fasttext/wiki.id.vec'}
Model trained with the hyperparameter 
 {'lr': 0.008, 'epoch': 10, 'wordNgrams': 4, 'dim': 300, 'pretrainedVectors': '/content/drive/MyDrive/College/Careers/Accepted Job/04 Solusi247 - Data Scientist/Jobs/fasttext/wiki.id.vec'}
2022-01-10 15:45:35.748218Training complete.{'lr': 0.008, 'epoch': 10, 'wordNgrams': 4, 'dim': 300, 'pretrainedVectors': '/content/drive/MyDrive/College/Careers/Accepted Job/04 Solusi247 - Data Scientist/Jobs/fasttext/wiki.id.vec'}
{'lr': 0.008, 'epoch': 10, 'wordNgrams': 4, 'dim': 300, 'pretrainedVectors': '/content/drive/MyDrive/College/Careers/Accepted Job/04 Solusi247 - Data Scientist/Jobs/fasttext/wiki.id.vec'},accuracy:0.9345454545454546,validation:0.7959183673469388

Model is quantized!!
([['__label__NEGATIVE', '__label__P

In [ ]:
# y_true
fmov=open('/content/test-sicepat.txt','r')
sentence_list=[]
label_list=[]
for f in fmov.readlines():
    lineinfo=f.split(' ')
    sentenceline=".".join(lineinfo[0:-1])
    sentence_list.append(sentenceline)
    label_list.append(str(lineinfo[0]).replace('\n',''))
    y_true = label_list

In [ ]:
# y_pred
y_pred = model.predict(processed_docs_test, k=1)
y_pred2 = list(y_pred)
y_pred2 = y_pred2.pop(0)
out = np.concatenate(y_pred2).ravel().tolist()
y_pred = out

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
print("confusion matrix: \n", confusion_matrix(y_true, y_pred))

# Accuracy
print("accuracy: ", accuracy_score(y_true, y_pred))

# Recall
from sklearn.metrics import recall_score
print("recall: \n", recall_score(y_true, y_pred, average=None))

# Precision
from sklearn.metrics import precision_score
print("precision: \n", precision_score(y_true, y_pred, average=None))

confusion matrix: 
 [[25  3  5]
 [ 2 30  0]
 [10  2 21]]
accuracy:  0.7755102040816326
recall: 
 [0.75757576 0.9375     0.63636364]
precision: 
 [0.67567568 0.85714286 0.80769231]
